In [87]:
pwd

'/Users/huyong/code/github/ZhihuLabelsPrediction/models/bigru_with_attention'

In [88]:
import pandas as pd

In [89]:
voc = pd.read_csv('./data/voc.csv')

In [90]:
voc[:10]

,word,word_id,word_frequency
0,<PAD>,0,0
1,<UNK>,1,0
2,，,2,4021192
3,的,3,3021942
4,。,4,1339001
5,？,5,1276040
6,是,6,1191386
7,我,7,1171798
8,不,8,1161837
9,了,9,930755


In [91]:
from tqdm import tqdm
word2id = {}
for index,row in tqdm(voc.iterrows()):
    word2id[row['word']] = row['word_id']

19606it [00:01, 10718.46it/s]


In [92]:
word2id['的']

3

In [93]:
import pickle

In [94]:
pickle.dump(word2id,open('./data/word2id.pkl','wb'))

In [95]:
topic_df = pd.read_csv('./data/topic_2_id.csv')
topic2id = {}
for index,row in tqdm(topic_df.iterrows()):
    topic2id[row['topic_name']] = row['topic_id']

25551it [00:02, 11351.72it/s]


In [96]:
topic2id['生活']

1

In [97]:
pickle.dump(word2id,open('./data/topic2id.pkl','wb'))

In [98]:
# 规整数据
df = pd.read_csv('./data/data_cut.csv')

In [99]:
df[:10]

,question_id,question_title,question_detail,tag_names,tag_ids
0,2600784210,如何 最 快 确定 自己 乘坐 的 飞机 自 几 号 航站楼 起飞 ？,有时 你 收到 的 航班 信息 里 没有 写明 是 哪 个 航站 楼 。 update . ...,飞机|生活|旅行|机场|城市生活|航空公司,264|1|27|1699|356|1267
1,1714848368,产品 假设 ： 用户 天生 善良 还是 邪恶 ？,美国 人 做 互联网 产品 ， 通常 会 假设 用户 天生 善良 ， 爱惜羽毛 ； 中国 人...,互联网用户|互联网产品,12183|291
2,1813305872,为什 么 知乎 的 用户 看起 来 更 喜欢 咖啡馆 而 不 是 茶馆 ？,NaN,生活方式|生活|城市生活,70|1|356
3,3776529318,回 家 给 父母 买什 么 礼物 最 好 ？,NaN,父母|春节|礼物,86|691|315
4,1805889373,SEO 的 业务 前景 如何 ？ 为什 么 这 么 认为 ？,NaN,搜索引擎营销（SEM）|网络营销|搜索引擎优化（SEO）,3225|683|805
5,3769324318,摩尔 庄园 受到 小朋友 欢迎 的 关键 原因 是 什 么 ？,NaN,摩尔庄园|淘米网,18575|21803
6,3776166802,对于 一个 国内 中小型 城市 来 说 ， 修建 市内 地铁 / 轻轨 是 否 是 解决 交...,NaN,城市规划|地铁|交通拥堵,542|501|9907
7,2227360902,阿 北 （ 杨勃 ） 在 豆瓣 早期 做出 的 最 重要 的 决定 有 哪 些 ？,NaN,豆瓣|著名公司早期关键决定|杨勃（阿北）,496|17383|15843
8,1004544321,中国 未 来 电子 商务 格局会 走向 何方 ？,从 美国 市场 看 ， 网络 零售 排名 前 十 位 中 有 8 个 是 实体 零售 企业 ...,电子商务,108
9,3811029348,关于 JavaScript 的 好 书 有 哪 些 ？,NaN,JavaScript|书籍推荐,259|147


In [100]:
import numpy as np

In [101]:
new_data = []
for index,row in tqdm(df.iterrows()):
    one_data = []
    question_title = row['question_title']
    one_data.append(len(question_title.split()))
    one_data.append([word2id.get(word,0) for word in question_title.split()])
    question_detail = row['question_detail']
    if question_detail is np.nan:
        one_data.append(0)
        one_data.append([])
    else:
        one_data.append(len(question_detail.split()))
        one_data.append([word2id.get(word,0) for word in question_detail.split()])
    topic_ids = row['tag_ids']
    one_data.append([int(topic_id) for topic_id in topic_ids.split('|')])
    new_data.append(one_data)

721608it [02:14, 5357.96it/s]


In [102]:
new_data[0]

[14,
 [35, 98, 328, 971, 32, 6245, 3, 1264, 373, 120, 539, 0, 6361, 5],
 25,
 [1159,
  33,
  1304,
  3,
  5339,
  452,
  75,
  42,
  0,
  6,
  51,
  34,
  0,
  1184,
  4,
  8058,
  64,
  7,
  61,
  337,
  80,
  5339,
  567,
  20,
  9],
 [264, 1, 27, 1699, 356, 1267]]

In [103]:
data_len = len(new_data)

In [104]:
import random

In [105]:
random.shuffle(new_data)

In [106]:
train_data = new_data[:int(data_len*0.9)]
dev_data = new_data[int(data_len*0.9):]

In [107]:
len(train_data)

649447

In [108]:
len(dev_data)

72161

In [109]:
pickle.dump(train_data,open('./data/train_data.pkl','wb'))

In [110]:
pickle.dump(dev_data,open('./data/dev_data.pkl','wb'))

In [111]:
len(topic2id)

25551

In [112]:
len(voc)

19606

In [85]:
voc = pd.read_csv('./data/voc.csv')

In [86]:
len(voc)

773121